## modules

In [29]:
import os
import json

import pandas as pd
import folium
from shapely.geometry import Point, Polygon

## import

In [6]:
df = pd.read_json('../data/filtered_data.json')

In [30]:
# Bezirksumringe
# Quelle: https://github.com/funkeinteraktiv/Berlin-Geodaten
base_dir = '/opt/pjs/urban_comparison/data/'
filename = 'berlin_bezirke.geojson'
with open(os.path.join(base_dir, filename), 'r') as infile:
    berlin_bezirke = json.load(infile)

## process

In [47]:
geojson_bezirke = {el['properties']['name']: Polygon(el['geometry']['coordinates'][0][0]) \
                   for el in berlin_bezirke['features']}

In [82]:
def get_bezirk(point, bezirke):
    for bezirk in geojson_bezirke:
        if geojson_bezirke[bezirk].contains(point):
            return bezirk
    return None

In [83]:
df['coords'] = df[['lon', 'lat']].apply(Point, axis=1)

In [84]:
df['bezirk'] = df['coords'].apply(lambda x: get_bezirk(x, geojson_bezirke))

## vis

In [168]:
map_obj = folium.Map(location = [52.5200, 13.4050], zoom_start = 10)

In [159]:
df['dummy'] = np.random.randint(1,10,df.shape[0])

In [160]:
df_agg = pd.DataFrame(df['bezirk'].value_counts()).reset_index()

In [161]:
df_agg = df_agg.rename(columns={'bezirk': 'count',
                       'index': 'bezirk'})

In [169]:
# generate choropleth map 
map_obj.choropleth(
    geo_data=berlin_bezirke,
    data=df_agg,
    columns=['bezirk', 'count'],
    key_on='feature.properties.name',
    fill_color='YlGnBu', 
    fill_opacity=1, 
    line_opacity=1,
    legend_name='count of amenities',
    smooth_factor=0)

In [170]:
map_obj